# Speech Emotion Recognition with MLP Classifier


# Dataset

The Ryerson Audio-Visual Database of Emotional Speech and Song(RAVDESS)

Audio-only Files
Audio-only Files of all Actors(01-24) are available as two separate zip files(~200MB each)

Speech file(Audio_Speech_Actors_01-24 zip,215MB) contains 1440 files :60 trials per actor x 24 actors= 1440 Song file
(Audio_Song_Actors_01-24 zip, 198 MB) contains 1012 files: 44 trials per actors x 23 actors =1012

Total=2452

In [1]:
!pip install librosa soundfile numpy sklearn pyaudio

  Using cached PyAudio-0.2.11.tar.gz (37 kB)
  Running setup.py clean for pyaudio
Failed to build pyaudio
    Running setup.py install for pyaudio: started
    Running setup.py install for pyaudio: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\ProgramData\Anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\KUNALR~1\\AppData\\Local\\Temp\\pip-install-087_q8uc\\pyaudio\\setup.py'"'"'; __file__='"'"'C:\\Users\\KUNALR~1\\AppData\\Local\\Temp\\pip-install-087_q8uc\\pyaudio\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\KUNALR~1\AppData\Local\Temp\pip-wheel-9zf4emd3'
       cwd: C:\Users\KUNALR~1\AppData\Local\Temp\pip-install-087_q8uc\pyaudio\
  Complete output (9 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.7
  copying src\pyaudio.py -> build\lib.win-amd64-3.7
  running build_ext
  building '_portaudio' extension
  error: Microsoft Visual C++ 14.0 is required. Get it with "Build Tools for Visual Studio": 

In [2]:
!pip install soundfile

# Making the necessary imports

In [3]:
import librosa
import soundfile
import os,glob,pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

Define a function extract_feature to extract the mfcc, chroma, and mel features from a sound file. This function takes 4 parameters- the file name and three Boolean Parameters for the three features:
 
mfcc- Mel Frequency Cepstral Coefficient, represents the short-term power spectrum of a sound

chroma- Pertains to the 12 different pitch classes 

mel- Mel Spectogram Frequency

In [4]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound File
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result=np.hstack((result,chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
            result=np.hstack((result,mel))
    return result

Now Let's define a dictionary to hold numbers and the Emotion available in the RAVDESS dataset and a list to hold all its emotions-neutral, calm, happy, sad, angry, fearfull, disgust, surprised

In [5]:
#DataFlair- Emotions in the RAVDESS dataset
emotions={
    '01':'neutral',
    '02':'calm',
    '03':'happy',
    '04':'sad',
    '05':'angry',
    '06':'fearfull',
    '07':'disgust',
    '08':'surprised'
}
#DataFlair- Emotions to Observe
observed_emotions=['calm','happy','fearfull','disgust']

# Load the data and extract features for each sound file

In [6]:
#DataFlair- Load the data and extract features from each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("C:\\Users\\KUNAL RANA\\Speech Emotion Recognition Project\\speech-emotion-recognition-ravdess-data\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

# Split the Dataset

Time to Split the dataset into training and testing sets! let's keep the test set 20% of everything and use the load_data function for this: 

In [7]:
#Dataflair - Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.20)

# Observe the shape of the training and testing datasets:

In [8]:
#DataFlair - Get the shape of the training and testing datasets 
print((x_train.shape[0], x_test.shape[0]))

(614, 154)


# Number of features extracted

In [9]:
#DataFlair - Get the number of Feature Extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


# MLP Classifier 

In [10]:
#DataFlair - Initialize the MultiLayer Perceptron Classifier
model = MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500) 

# Fit/train the model

In [11]:
#DataFlait - Train the model
model.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

# Predict the accuracy of our model

Let's predict the values for the test set. This give us y-pred(the predicted emotions for the features in the test set)

In [12]:
y_pred = model.predict(x_test) 

To calculate the accuracy of our model we'll call up the accuracy score() function we imported from sklearn. Finally we'll  round the accuracy to 2 decimal places and print it out

In [13]:
#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 74.68%


# Classification Report

In [15]:
from sklearn.metrics import classification_report

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

        calm       0.78      0.86      0.82        44
     disgust       0.80      0.68      0.74        41
    fearfull       0.68      0.76      0.71        33
       happy       0.73      0.67      0.70        36

    accuracy                           0.75       154
   macro avg       0.74      0.74      0.74       154
weighted avg       0.75      0.75      0.75       154



# Confusion Matrix

In [16]:
from sklearn.metrics import confusion_matrix

matrix=confusion_matrix(y_test,y_pred)
print(matrix)

[[38  2  4  0]
 [ 6 28  4  3]
 [ 1  1 25  6]
 [ 4  4  4 24]]


# Thank You